# Task 1_Basic_Tello_Control

## *1.Taking_off and landing*

    "write a short code to command the drone to
    hover in the air a few seconds and then land."

#### STEP_0 : Preparation
    1. Install tello API library with pip3

        ```bash
            pip3 install djitellopy2
        ```

#### STEP_1 : import tello drone library

In [ ]:
from djitellopy import Tello
from time import sleep

#### STEP_2 : Create drone instance and connect to drone

In [ ]:
tello = Tello()
tello.connect()

#### STEP_3 : print current battery level

In [ ]:
print("BATTERY:", tello.get_battery())

#### STEP_4 : Sending take off command and hover for 3 second

In [ ]:
tello.takeoff()
sleep(3)

#### STEP_5 : Sending land command

In [ ]:
tello.land()

## *2.Moving_around*

    "write a short code to command the drone into moving
    following a 2*2m square(approximately based on
    flight time at constant speed)"

#### STEP_1 : import library and connect to drone

In [ ]:
from djitellopy import Tello

tello = Tello()
tello.connect()
print("BATTERY:", tello.get_battery())

tello.set_speed(20)

#### STEP_2 : taking off and moving up 50 cm

In [ ]:
tello.takeoff()
tello.move_up(50)

#### STEP_3 : forward 100 cm and back origin position

In [ ]:
tello.move_forward(100)
tello.move_back(100)

#### STEP_4 : left 100 cm and back origin position

In [ ]:
tello.move_left(100)
tello.move_right(100)

#### STEP_5 : backward 100 cm and back origin position

In [ ]:
tello.move_back(100)
tello.move_forward(100)

#### STEP_6 : right 100 cm and back origin position then landing

In [ ]:
tello.move_right(100)
tello.move_left(100)
tello.land()

![Flight_Path](resource/move_around_path.JPG)

## *3.Monitoring the battery level*
    "write a short code that commands the drone to land
    if the battery level is below a threshold. Also,
    print the battery level in the terminal when it 5% steps."

#### STEP_1 : Initialization Drone

In [ ]:
from djitellopy import Tello
import cv2

# frame size
w, h = 360, 240

# create tello instance
tello = Tello()
tello.connect()
tello.streamon()

# print current battery level
print("BATTERY:", tello.get_battery())

#### STEP_2 : Set a start count so that drone only takeoff once


In [ ]:
# zero mean ready to flight, one mean has flight
startCounter = 0

#### STEP_3 : Monitoring battery level in super loop

In [ ]:
while True:
    # take off at first time
    if startCounter == 0:
        tello.takeoff()
        startCounter = 1
    # Get stream frame and resize frame
    img = tello.get_frame_read().frame
    img = cv2.resize(img, (360, 240))
    # put battery level on each frame
    level = tello.get_battery()
    cv2.putText(img, str(level), (10, 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 1)
    cv2.imshow("Tello", img)
    # wait frame loading and press 'q' to land
    if cv2.waitKey(1) & 0xFF == ord('q'):
        tello.land()
        break


## *4.Tello_acrobatics*
    "use your keyboard input to control the drone. Use keys for
    (1)taking off, (2)landing, and (3)-(8) doing flips
     in different directions"

#### STEP_0 : Create a windows with pygame

In [ ]:
import pygame

def init():
    pygame.init()
    win = pygame.display.set_mode((400, 400))

def getKey(keyName):
    ans = False
    for eve in pygame.event.get(): pass
    keyInput = pygame.key.get_pressed()
    myKey = getattr(pygame, 'K_{}'.format(keyName))
    if keyInput[myKey]:
        ans = True
    pygame.display.update()
    return ans

def main():
    if getKey("LEFT"):
        print("Left key Pressed")
    if getKey("RIGHT"):
        print("Right key Pressed")


if __name__ == '__main__':
    init()
    while True:
        main()

#### STEP_1 : Initialization Drone

In [ ]:
from djitellopy import Tello
from time import sleep
from Task1.resource import KeyPressModule as kp
import cv2
from threading import Thread

# py game initialization
kp.init()
# tello initialization
tello = Tello()
tello.connect()
print("BATTERY:", tello.get_battery())

#### STEP_2 : Define Keyboard behavior

In [ ]:
def getKeyboardInput():
    lr, fb, ud, yv = 0, 0, 0, 0
    speed = 50
    if kp.getKey("LEFT"):
        lr = -speed
    elif kp.getKey("RIGHT"):
        lr = speed
    if kp.getKey("UP"):
        fb = speed
    elif kp.getKey("DOWN"):
        fb = -speed
    if kp.getKey("w"):
        ud = speed
    elif kp.getKey("s"):
        ud = -speed
    if kp.getKey("a"):
        yv = -speed
    elif kp.getKey("d"):
        yv = speed
    if kp.getKey("q"):
        tello.land()
        # sleep(3)
    if kp.getKey("e"):
        tello.takeoff()
    if kp.getKey("i"):
        tello.flip_forward()
    if kp.getKey("k"):
        tello.flip_back()
    if kp.getKey("j"):
        tello.flip_left()
    if kp.getKey("l"):
        tello.flip_right()
    if kp.getKey("x"):
        print("HEIGHT:", tello.get_height())
        print("BAROMETER:", tello.get_barometer())

    return [lr, fb, ud, yv]

#### STEP_4 : Put video stream in external thread

In [ ]:
def stream():
    tello.streamon()
    # get video stream
    while True:
        level = tello.get_battery()
        img = tello.get_frame_read().frame
        img = cv2.resize(img, (360, 240))
        cv2.putText(img, str(level), (10, 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 1)
        cv2.imshow("Tello", img)
        cv2.waitKey(1)

#### STEP_5 : Main Thread

In [ ]:
'''
Press E to take off, Q to landing
A and D control drone's yaw (clockwise or counter clockwise)
W and S control drone's up and down
Arrow UP and DOWN control drone's forward and backward
Arrow RIGHT and LEFT control drone's left and right
I, K, J, L doing flip
'''
if __name__ == "__main__":
    video_thread = Thread(target=stream)
    video_thread.start()
    while True:
        vals = getKeyboardInput()
        tello.send_rc_control(vals[0], vals[1], vals[2], vals[3])
        sleep(0.05)